In [1]:
import src.pmcmc.main as pmcmc_main

In [2]:
pmcmc_main.main('06', '2024-04-27')

PMCMC Progress: 0it [00:00, ?it/s]                                        1.51s/it]


Array([[0.001     , 0.0160432 , 0.01137184, ..., 0.10509229, 0.10826205,
        0.10643896],
       [0.03911028, 0.02117727, 0.01048695, ..., 0.11249776, 0.10198469,
        0.10087529],
       [0.03890042, 0.00979877, 0.01719936, ..., 0.13215451, 0.13641371,
        0.09556162],
       ...,
       [0.0077249 , 0.03675001, 0.01247766, ..., 0.1325028 , 0.08701457,
        0.11388317],
       [0.01654181, 0.02720235, 0.02271833, ..., 0.128433  , 0.09967344,
        0.09053642],
       [0.0024256 , 0.03214093, 0.02477897, ..., 0.10575061, 0.13215952,
        0.09792652]], dtype=float32)

In [3]:
pmcmc_main.main('04', '2024-04-27')

PMCMC Progress: 0it [00:00, ?it/s]                                        1.49s/it]


Array([[0.001     , 0.01413135, 0.00945999, ..., 0.11513677, 0.11830653,
        0.11648345],
       [0.03911028, 0.01926542, 0.01370917, ..., 0.11888187, 0.11202918,
        0.12203167],
       [0.03890042, 0.00788692, 0.00390901, ..., 0.12524697, 0.12314107,
        0.12669365],
       ...,
       [0.0077249 , 0.03675001, 0.01577767, ..., 0.08991994, 0.08591162,
        0.10657184],
       [0.01654181, 0.02720235, 0.03227675, ..., 0.09493215, 0.09701283,
        0.08301048],
       [0.0024256 , 0.03214093, 0.03067963, ..., 0.10973269, 0.10938941,
        0.10764249]], dtype=float32)

In [4]:
import os
import numpy as np 
import pandas as pd
from src.utils import paths

locations = ['04', '06']
target_dates = pd.read_csv('../datasets/target_dates.csv')

In [7]:
dates = target_dates['date'].values

In [8]:
for loc in locations:
    print("Working location ", loc)
    base_dir = os.path.join(paths.OUTPUT_DIR, 'pmcmc_runs')
    loc_dir = os.path.join(base_dir, loc)
    beta_file_path = os.path.join(loc_dir, 'mle_betas_20241020.csv')
    df = pd.read_csv(beta_file_path)
 
    # Average over the particles
    row_means = df.mean(axis=0)
    # Transpose the DataFrame
    transposed_df = row_means.to_frame() 
    num_rows = len(transposed_df)
    end_date = pd.to_datetime("2024-04-27")
    date_range = pd.date_range(end=end_date, periods=num_rows)
    transposed_df["date"] = date_range
    transposed_df.rename({0: "beta"}, axis="columns", inplace=True)
    
    df = transposed_df
    for target_date in target_dates["date"]:
        start_date = pd.to_datetime(target_date) - pd.Timedelta(days=50)
        filtered_df = df[(df["date"] > start_date) & (df["date"] <= target_date)]
        filtered_df.insert(0, "time_0", range(len(filtered_df)))
        filtered_df = filtered_df.drop(columns=["date"])
        file_path = os.path.join(paths.PF_OUTPUT_DIR, target_date, f"{loc}.csv")
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        filtered_df.to_csv(file_path, index=False)

Working location  04
Working location  06
